# Исследование надёжности заёмщиков

Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
from nltk.stem import SnowballStemmer

In [2]:
clientele = pd.read_csv('/datasets/data.csv')

In [3]:
clientele.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Не все названия столбцов  в достаточной степени отражают их суть, произведем замену:

'dob_years' -> 'age' (возраст клиента в годах)

'debt' -> 'loan_delinquency' (имел ли задолженность по возврату кредитов)

'total_income' -> 'monthly_income' (ежемесячный доход)

In [4]:
clientele = clientele.rename(columns={'dob_years':'age', 'debt':'loan_delinquency','total_income':'monthly_income'})
clientele.head(10)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
clientele.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
age                 21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
loan_delinquency    21525 non-null int64
monthly_income      19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из краткой сводки данных следует, что общее количество клиентов в базе 21525. 
Однако, трудовой стаж в днях записан лишь для 19351, аналогичная ситуация с ежемесячным доходом, т.е. данные отсутствуют в столбцах 'days_employed' и 'monthly_income'.
Необходимо выяснить, существует ли зависимость между отсутствием данных в этих столбцах.
Возможно, имеет смысл заменить тип данных в столбце 'days_employed' на int - дни лучше измерять целочисленно, и это поможет сэкономить память.
За исключением вышеописанного, данные подозрений не вызывают. Воспользуемся статистическим анализом (метод describe) для более подробной оценки.

In [6]:
clientele.describe()

,children,days_employed,age,education_id,family_status_id,loan_delinquency,monthly_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Столбец 'children':
Минимальное значение - (-1). Возможно, информация отсутствует, или минус добавлен случайно.
Максимальное значение - (20). Стоит проверить. Может быть это просто 2 и ошибка случайна, а может и уникальный случай.

Столбец 'days_employed':
Среднее значение - (63047/365=173). 173 года - многовато. Совсем не похоже на правду.
Максимальное значение - (401755/365=1100). 1100 лет - выглядит абсурдно, даже если учесть високосные года, эта цифра не будет отражать действительность. 
В столбце множество отрицательных значений. Есть вероятность, что это недочет, и значения можно будет просто взять по модулю.

Столбец 'age':
Минимальный возраст - (0). Вероятно, данные просто не были внесены в таблицу (трудно представить ситуацию, в которой заемщик не указал возраст).


Приступим к проверке.
Существует ли связь между отсутствием данных в столбцах 'days_employed' и 'monthly_income'?

In [7]:
clientele[(clientele['days_employed'].isnull() == True) & (clientele['monthly_income'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
age                 2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
loan_delinquency    2174 non-null int64
monthly_income      0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Связь существует. Если данные отсутсвуют в 'days_employed', то в 'monthly_income' их тоже нет. 
Возможно, отсутсвие данных связано с типом занятости: как правило, от него зависит стаж и зарплата. 

In [8]:
clientele[(clientele['days_employed'].isnull() == True) & (clientele['monthly_income'].isnull() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Взаимосвязь отсутсвует, а это значит, что мы не можем заполнить пустные строки исходя из среднего. Ради эксперимента возьмем другой столбец и попробуем отыскать какую-нибудь случайную зависимость, возможно, ошибку при переносе данных и т.п.

In [9]:
clientele[(clientele['days_employed'].isnull() == True) & (clientele['monthly_income'].isnull() == True)]['gender'].value_counts()

F    1484
M     690
Name: gender, dtype: int64

Взаимосвязь снова отсутствует. Значит заполнять пропуски придется при помощи расчета медианы для каждой группы, взятой по некоторому параметру. 

Разберемся со столбцом 'children'.

In [10]:
print('Количество людей с -1 ребенком:',clientele[clientele['children'] == -1].count()[0])
print('Количество людей с 20 детьми:',clientele[clientele['children'] == 20].count()[0])
print('Количество уникальных людей с 20 детьми:',len(clientele[clientele['children'] == 20]['days_employed'].unique()))

Количество людей с -1 ребенком: 47
Количество людей с 20 детьми: 76
Количество уникальных людей с 20 детьми: 68


Имеем:
Строки с -1 ребенком - исправить.
Строки с 20 детьми - не является уникальным случаем, эту ошибку тоже необходимо исправить. (Возможно, верное значение 2)

Разберемся со столбцом 'days_employed'.

In [11]:
print('Количество строк, в которых общий трудовой стаж > 0:',clientele[clientele['days_employed'] > 0].count()[0])
print('Количество строк, в которых общий трудовой стаж < 0:',clientele[clientele['days_employed'] < 0].count()[0])
clientele_retiree = clientele[(clientele['income_type'] == 'пенсионер')]
print('Средний возраст пенсионеров: {0:.0f}' .format(clientele_retiree['age'].mean()))
print('Среднее количество отработанных пенсионерами лет: {0:.0f}'.format(clientele_retiree['days_employed'].mean()/365))
print('Медианное значение отработанных пенсионерами лет: {0:.0f}'.format(clientele_retiree['days_employed'].median()/365))

Количество строк, в которых общий трудовой стаж > 0: 3445
Количество строк, в которых общий трудовой стаж < 0: 15906
Средний возраст пенсионеров: 59
Среднее количество отработанных пенсионерами лет: 1000
Медианное значение отработанных пенсионерами лет: 1001


Процент строк, в которых стаж имеет отрицательное значение, около 80%. Это - явная ошибка. 

К тому же, например, среднее количество лет стажа пенсионеров в 17 раз превышает их средний возраст, что вряд ли может быть правдой. Может быть среднее не подходит для оценки? На всякий случай попробуем медиану. В общем, результат не улучшился.

Внесем предположение, что при слишком больших значениях стажа есть вероятность, что он указан в часах.

In [12]:
print('Среднее количество отработанных пенсионерами лет (стаж был в часах): {0:.0f}'.format(clientele_retiree['days_employed'].mean()/365/24))
print('Медианное значение отработанных пенсионерами лет (стаж был в часах): {0:.0f}'.format(clientele_retiree['days_employed'].median()/365/24))

Среднее количество отработанных пенсионерами лет (стаж был в часах): 42
Медианное значение отработанных пенсионерами лет (стаж был в часах): 42


Такие значения больше похожи на правду.

Разберемся со столбцом 'age'.

In [13]:
print("Количество людей с нулевым возрастом:", clientele[clientele['age'] == 0].count()[0])

Количество людей с нулевым возрастом: 101


В качестве финального пункта изучения данных, рассмотрим уникальные значения в каждом столбце, чтобы окончателно выявить все недочеты датафрейма.

In [14]:
for row in clientele: 
  print(clientele[row].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
-986.927316     1
-7026.359174    1
-4236.274243    1
-6620.396473    1
-1238.560080    1
               ..
-2849.351119    1
-5619.328204    1
-448.829898     1
-1687.038672    1
-582.538413     1
Name: days_employed, Length: 19351, dtype: int64
35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: age, dtype: int64
среднее         

Выводы:

Столбец "children". Ошибочные значения (20), (-1). Необходимо произвести замену.

Столбец "days_employed". Пропущенные значения (2174). Заполнить пропуски. Положительные и отрицательные значения. Тип данных заменить на Integer.

Столбец "age". Люди с нулевым возрастом. Вероятно, данные просто не были внесены, необходимо исправить.

Столбец "education". Дубликаты в названиях, различающиеся регистром. Привести к одному регистру.

Столбец "family_status". Для типа 'Не женат / не замужем' имеет смысл поправить регистр, чтобы везде были строчные буквы.

Столбец "monthly_income". Пропущенные значения (2174). Тип данных заменить на Integer.

Столбец "purpose". Много дубликатов с похожими или идентичными причинами. Требуется объеденить в одинаковые значения.

## Шаг 2. Предобработка данных

### Обработка пропусков

Столбец "children".

In [15]:
print("Количество строк с 'children' -1:", clientele[clientele['children'] == -1].count()[0])
print("Количество строк с 'children' 0:", clientele[clientele['children'] == 0].count()[0])
print("Количество строк с 'children' 20:", clientele[clientele['children'] == 20].count()[0])
print("Количество строк с 'children' 2:", clientele[clientele['children'] == 2].count()[0])
print("Количество строк с 'children' > 2:", clientele[clientele['children'] > 2].count()[0])

Количество строк с 'children' -1: 47
Количество строк с 'children' 0: 14149
Количество строк с 'children' 20: 76
Количество строк с 'children' 2: 2055
Количество строк с 'children' > 2: 456


Данная часть информации понадобится нам для ответа на один из четырах вопросов задания, даные нужны как можно точнее, поэтому проведем небольшой анализ.

При замене (-1) на 0, с учетом количества строк, статистика сильно не пострадает. Количество строк со значением 0 увеличится всего на 0,3%. 

А вот в случае с заменой 20 на 2, количество строк увеличится на 3,7%, что более существенно. Тогда допустим вариант, в котором строки со значением 20 будут отнесены в группу 'многодетные' (>2 детей), но при этом количество строк увеличится на 16,6% - совсем плохо. 

С учетом вышесказанного, имеет смысл заменить 20 на 2: процент изменения меньше, а то, что при внесении данных случайно был добавлен лишний 0, выглядит более вероятным.

In [16]:
clientele['children'] = clientele['children'].replace(20, 2)
clientele['children'] = clientele['children'].replace(-1, 0)

Столбец "days_employed".

In [17]:
print(clientele['age'].max())

75


Как правило, принцип заполнения отсутствующих значений должен отвечать определенной цели. В ТЗ нет вопросов, которые были бы как-то связаны с этой категорией данных, поэтому определить, как и чем заполнить строки, затруднительно. 
Исходя из этого, самый оптимальный вариант - потренироваться заполнять пустые строки, используя полученные ранее навыки.
Пропущенные строки в данном столбце могли появиться потому, что некоторые клиенты не указали в анкете стаж, а возможно это - ошибка заполнения таблицы, или некорректный перенос данных с различных носителей. Помимо этого ранее было внесено предложение, что при аномально больших значениях стаж был указан не в днях, а в часах.

Стаж логичнее всего связывать с возрастом, но уникальных значений для возраста в таблице слишком много. Это значит, что необходимо объеденить значения в группы для более удобного заполнения пустых строк.
Проведем деление на группы, зная максимальное значения возраста.

Оптимальное число групп - 4:

0 - 30

31 - 45

46 - 60

61 - 75


In [18]:
def days_employed(row):
    age = row['age']
    
    if age <= 30:
        return '19 - 30 лет'

    if age > 30 and age <= 45:
        return '31 - 45 лет'

    if age > 45 and age <= 60:
        return '46 - 60 лет'

    if age > 60 and age <= 75:
        return '61 - 75 лет'


Создадим новый столбец 'employers_group', который будет нести в себе информацию о том, к какой группе относится каждая строка.

In [19]:
clientele['age_category'] = clientele.apply(days_employed, axis = 1)
clientele['days_employed'] = clientele['days_employed'].fillna(0)
clientele.isnull().sum() 

children               0
days_employed          0
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
loan_delinquency       0
monthly_income      2174
purpose                0
age_category           0
dtype: int64

Теперь заполним пропуски для каждой группы в 'employers_group'.

In [20]:
clientele.loc[(clientele['age_category'] == '19 - 30 лет') & (clientele['days_employed'] == 0 ), 'days_employed'] = clientele.groupby('age_category')['days_employed'].mean()[0]
clientele.loc[(clientele['age_category'] == '31 - 45 лет') & (clientele['days_employed'] == 0 ), 'days_employed'] = clientele.groupby('age_category')['days_employed'].mean()[1]
clientele.loc[(clientele['age_category'] == '46 - 60 лет') & (clientele['days_employed'] == 0 ), 'days_employed'] = clientele.groupby('age_category')['days_employed'].mean()[2]
clientele.loc[(clientele['age_category'] == '61 - 75 лет') & (clientele['days_employed'] == 0 ), 'days_employed'] = clientele.groupby('age_category')['days_employed'].mean()[3]

In [21]:
clientele.head(30)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,31 - 45 лет
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,31 - 45 лет
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,31 - 45 лет
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,31 - 45 лет
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,46 - 60 лет
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,19 - 30 лет
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,31 - 45 лет
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,46 - 60 лет
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,31 - 45 лет
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,31 - 45 лет


Теперь разберемся с аномально большими значениями стажа, исходя из соображений о том, что они указаны не в днях, а в часах.

Создадим функцию, которая будет возвращать определенное значение, в зависимости от того, относится ли рассматриваемая строка к строкам с аномальными значениями. С дальнейшей заменой данных на откорректированые.

In [22]:
def days_employed_hours(row):
    age = row['age']
    days_employed = row['days_employed']
    
    if (days_employed/365) >= age:
        return days_employed/24
    if (days_employed/365) < age:
        return days_employed
        
clientele['days_employed'] = clientele.apply(days_employed_hours, axis = 1)
clientele['days_employed'] = clientele['days_employed'].fillna(0)
clientele.describe()

,children,days_employed,age,education_id,family_status_id,loan_delinquency,monthly_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.477538,1015.641917,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755467,6778.383622,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2518.168900,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-982.531720,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,1105.335810,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,16739.808353,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Выглядит правдоподобно. Теперь средний стаж составляет 9,6 лет, а максимальный около 46 лет.
Однако, если обратить внимание на минимальное значение, то можно заметить, что по модулю оно несколько больше максимального.
Поэтому теперь необходимо поработать с отрицательными значениями.

Разберемся с отрицательными значениями. 

Воспользуемся функцией agg(), которая значительно упрощает суммирование данных и функцией dict() для создать словаря с заранее подготовленным набором данных. 

In [23]:
clientele_grouped = clientele.groupby('income_type').agg({'days_employed':['count', lambda x: sum(x>0)]})
clientele_grouped_re = dict(zip(clientele_grouped.columns.levels[1], ['Кол-во строк', 'Кол-во строк со значением > 0']))

clientele_grouped = clientele_grouped.rename(columns=clientele_grouped_re, level=1)

clientele_grouped

days_employed                              
                 Кол-во строк Кол-во строк со значением > 0
income_type                                                
безработный                 2                           2.0
в декрете                   1                           0.0
госслужащий              1459                         147.0
компаньон                5085                         508.0
пенсионер                3856                        3856.0
предприниматель             2                           1.0
сотрудник               11119                        1105.0
студент                     1                           0.0

В случае с 'безработными' и 'пенсионерами' все строки имеют положительные значения. В малочисленных категориях положительные строки составляют от 50% ('предприниматель') до 0% ('в декрете', 'студент'). А в многочисленных категориях процент положительных строк около 10%. 


В целом, в ТЗ нет вопросов, как-либо связанных с этой частью данных, поэтому, вероятно, менять их необязательно - в нашем случае они не будут использованы для ответа на поставленные вопросы. Однако, все равно хотелось бы привести таблицу к приятному виду - применим abs(). Что касается причин возникновения: можно предположить, что отрицательные значения - ошибка ввода.    

In [24]:
clientele['days_employed'] = clientele['days_employed'].abs()
clientele.head(15)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,31 - 45 лет
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,31 - 45 лет
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,31 - 45 лет
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,31 - 45 лет
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,46 - 60 лет
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,19 - 30 лет
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,31 - 45 лет
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,46 - 60 лет
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,31 - 45 лет
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,31 - 45 лет


Столбец "age".

Для того чтобы заполнить строки со значением 0 посчитаем медиану возраста клиента в годах для каждого из типов занятости.

In [25]:
income_type_gr = clientele.groupby('income_type')['age'].median()
income_type_gr

income_type
безработный        38.0
в декрете          39.0
госслужащий        40.0
компаньон          39.0
пенсионер          60.0
предприниматель    42.5
сотрудник          39.0
студент            22.0
Name: age, dtype: float64

Произведем замены для строк с нулевым возрастом исходя из медиан возрастов клиентов для каждого типа занятости. 

In [26]:
clientele.loc[(clientele['income_type'] == 'безработный') & (clientele['age'] == 0), 'age'] = 38
clientele.loc[(clientele['income_type'] == 'в декрете') & (clientele['age'] == 0), 'age'] = 39
clientele.loc[(clientele['income_type'] == 'госслужащий') & (clientele['age'] == 0), 'age'] = 40
clientele.loc[(clientele['income_type'] == 'компаньон') & (clientele['age'] == 0), 'age'] = 39
clientele.loc[(clientele['income_type'] == 'пенсионер') & (clientele['age'] == 0), 'age'] = 60
clientele.loc[(clientele['income_type'] == 'предприниматель') & (clientele['age'] == 0), 'age'] = 43
clientele.loc[(clientele['income_type'] == 'сотрудник') & (clientele['age'] == 0), 'age'] = 39
clientele.loc[(clientele['income_type'] == 'студент') & (clientele['age'] == 0), 'age'] = 22

Проверка:

In [27]:
print("Количество строк с 'age' 0 -", clientele[clientele['age'] == 0].count()[0])

Количество строк с 'age' 0 - 0


Столбец "education".

Откорректируем регистр.

In [28]:
clientele['education'] = clientele['education'].str.lower()
print(clientele['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Столбец "family_status".

Откорректируем регистр.

In [29]:
clientele['family_status'] = clientele['family_status'].str.lower()
print(clientele['family_status'].unique())

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


Столбец "monthly_income".

Данные о ежемесячном доходе будут одним из критириев оценки надежности заемщиков, поэтому для более точного анализа используем медиану. В данном столбце лучше всего использовать связь: Тип занятости - Ежемесячный доход. Такой подход выглядит наиболее целесообразным, ведь от занятости зависит зарплата, к тому же, при таком разделении мы получим данные сразу по 8 группам, что, возможно, повысит точность.
Пропуски в этой части связаны с пропусками в столбце, который содержит информацию о стаже. В таком случае причинины похожи: клиент не оставил информцию ни о стаже, ни о доходе; некорректное заполнение таблицы; ошибки при переносе данных с различных носителей.

Исследуем медианы дохода для каждого из типов занятости и проведем замену.

In [30]:
monthly_income_gr = clientele.groupby('income_type')['monthly_income'].median()
monthly_income_gr

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: monthly_income, dtype: float64

In [31]:
clientele['monthly_income'] = clientele['monthly_income'].fillna(0)

clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'безработный'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[0]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'в декрете'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[1]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'госслужащий'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[2]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'компаньон'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[3]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'пенсионер'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[4]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'предприниматель'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[5]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'сотрудник'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[6]
clientele.loc[(clientele['monthly_income'] == 0) & (clientele['income_type'] == 'студент'), 'monthly_income'] = clientele.groupby('income_type')['monthly_income'].mean()[7]

**Вывод**

In [32]:
clientele.describe()

,children,days_employed,age,education_id,family_status_id,loan_delinquency,monthly_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.477538,4493.186300,43.496167,0.817236,0.972544,0.080883,1.656844e+05
std,0.755467,5175.751141,12.231538,0.548138,1.420324,0.272661,9.797844e+04
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1024.652083,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2143.902066,43.000000,1.000000,0.000000,0.000000,1.453424e+05
75%,1.000000,5068.532819,53.000000,1.000000,1.000000,0.000000,1.955499e+05
max,5.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [33]:
clientele.head(15)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,31 - 45 лет
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,31 - 45 лет
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,31 - 45 лет
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,31 - 45 лет
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,46 - 60 лет
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,19 - 30 лет
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,31 - 45 лет
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,46 - 60 лет
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,31 - 45 лет
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,31 - 45 лет


Все пропуски были заполнены на основании уже существующих в таблице данных. Некоторые некорректные значения, явно не отражающие действительность, были откорректированы.

### Замена типа данных

Исходя из общей информации о таблице, мы видим, что столбцы со стажем и ежемесячным доходом, имеют вещественный тип данных, который хорошо бы заменить на целочисленный. Дни логичнее всего измерять в целых числах, а в доходах нам нет необходимости хранить такое количество чисел после запятой. Замена типа данных поможет нам сэкономить память.

In [34]:
clientele['days_employed'] = clientele['days_employed'].astype('int')
clientele['monthly_income'] = clientele['monthly_income'].astype('int')

In [35]:
clientele.dtypes

children             int64
days_employed        int64
age                  int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
loan_delinquency     int64
monthly_income       int64
purpose             object
age_category        object
dtype: object

**Вывод**

Типы данных для 'days_employed' и 'monthly_income' были изменены на Integer. Остальные типы соответсвуют данным в столбцах.

### Обработка дубликатов

Необходимо узнать количество дубликатов в наборе данных, используем стандартный метод duplicated(). Для вывода суммируем.

In [36]:
clientele.duplicated().sum()

71

Выведем таблицу, чтобы посмотреть данные и оценить возможные причины возникновения, после чего удалим дубликаты.

In [37]:
clientele[clientele.duplicated(keep = False)].sort_values(by = ['days_employed', 'children', 'age'])

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category
15892,0,1105,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,145342,сделка с подержанным автомобилем,19 - 30 лет
19321,0,1105,23,среднее,1,не женат / не замужем,4,F,сотрудник,0,145342,сделка с подержанным автомобилем,19 - 30 лет
3452,0,1105,29,высшее,0,женат / замужем,0,M,сотрудник,0,145342,покупка жилой недвижимости,19 - 30 лет
18328,0,1105,29,высшее,0,женат / замужем,0,M,сотрудник,0,145342,покупка жилой недвижимости,19 - 30 лет
4216,0,1105,30,среднее,1,женат / замужем,0,M,сотрудник,0,145342,строительство жилой недвижимости,19 - 30 лет
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,10790,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,122440,операции со своей недвижимостью,61 - 75 лет
9528,0,10790,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,122440,операции со своей недвижимостью,61 - 75 лет
6537,0,10790,71,среднее,1,гражданский брак,1,F,пенсионер,0,122440,на проведение свадьбы,61 - 75 лет
7938,0,10790,71,среднее,1,гражданский брак,1,F,пенсионер,0,122440,на проведение свадьбы,61 - 75 лет


In [38]:
clientele = clientele.drop_duplicates()

Проверка.

In [39]:
clientele.duplicated().sum()

0

**Вывод**

Из датафрейма были удалены все дубликаты. Это важно, потому как они могли сместить финальные значения для выводов. Их возникновение, скорее всего, связано с ошибкой вводы данных. Стоит отметить, что некоторые значения в таблице были заполнены в ходе работы исходя из средних значений, что тоже могло привести к появлению дубликатов. 

### Лемматизация

Все необходимые библиотеки были импортированы в самом начале. 
В столбце 'purpose', который содержит информацию о целях кредита и является важным источником информации для анализа, часть данных представляет собой схожие цели, описанные разными словами.
Для исправления ситуации необходимо провести лемматизацию, чтобы выяснить, какие слова и как часто используются для описания целей кредита.

Для начала получим список уникальных лемм. После чего посчитаем, сколько раз используется каждое слово, и выведем список таким образом, чтобы видеть наиболее часто употребляемые леммы.

In [40]:
m = Mystem()
def lemma_purpose(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

clientele['purpose_word'] = clientele['purpose'].apply(lemma_purpose)
    
unique_lemmas = Counter(clientele['purpose_word'])
sorted(unique_lemmas.items(), key = lambda pair: pair[1], reverse=True)

[('автомобиль \n', 972),
 ('свадьба \n', 791),
 ('на   проведение   свадьба \n', 768),
 ('сыграть   свадьба \n', 765),
 ('операция   с   недвижимость \n', 675),
 ('покупка   коммерческий   недвижимость \n', 661),
 ('операция   с   жилье \n', 652),
 ('покупка   жилье   для   сдача \n', 651),
 ('операция   с   коммерческий   недвижимость \n', 650),
 ('покупка   жилье \n', 646),
 ('жилье \n', 646),
 ('покупка   жилье   для   семья \n', 638),
 ('строительство   собственный   недвижимость \n', 635),
 ('недвижимость \n', 633),
 ('операция   со   свой   недвижимость \n', 627),
 ('строительство   жилой   недвижимость \n', 624),
 ('покупка   недвижимость \n', 621),
 ('покупка   свой   жилье \n', 620),
 ('строительство   недвижимость \n', 619),
 ('ремонт   жилье \n', 607),
 ('покупка   жилой   недвижимость \n', 606),
 ('на   покупка   свой   автомобиль \n', 505),
 ('заниматься   высокий   образование \n', 496),
 ('сделка   с   подержанный   автомобиль \n', 486),
 ('на   покупка   подержать   авт

В полученном списке можно увидеть, в каком контексте употреблялись слова, какие формулировки самые популярые и сколько раз они встречались.
Из основного можно выделить категории:

Жилье.
Автомобиль.
Образование.
Свадьба.

Для более детального анализа имеет смысл разделить жилье на три подкатегории:
коммерчиская недвижимость, жилье и ремонт жилья. Возможно, по каждой из этих категорий суммы займа сильно различны.

In [41]:

clientele

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category,purpose_word
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,31 - 45 лет,покупка жилье \n
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,31 - 45 лет,приобретение автомобиль \n
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,31 - 45 лет,покупка жилье \n
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,31 - 45 лет,дополнительный образование \n
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,46 - 60 лет,сыграть свадьба \n
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,31 - 45 лет,операция с жилье \n
21521,0,14330,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,61 - 75 лет,сделка с автомобиль \n
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,31 - 45 лет,недвижимость \n
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,31 - 45 лет,на покупка свой автомобиль \n


**Вывод**

В качестве результата лемматизации и последющей обработки, мы получили список целей, который используем для создания столбца с категориями.
Причина, по которой данные были представлены в таком виде, скорее всего, заключается в том, что цель кредита указывалась в анкете в свободной форме и в ней же была занесена в датафрейм.

### Категоризация данных

Начнем с создания нового столбца 'purpose_category', в котором на основе результатов лемматизации присвоим каждой цели кредита категорию.

In [42]:

def purpose_category(list):
    if 'автомобиль' in list:
        return "автомобиль"
    if "образование" in list:
        return "образование"
    if "свадьба" in list:
        return "свадьба"
    if "недвижимость" and 'коммерческий' in list:
        return "коммерческая недвижимость"
    if 'ремонт' in list:
        return 'ремонт'
    if "жилье" or "недвижимость" in list:
        return "жилье"

In [43]:
clientele['purpose_category'] = clientele['purpose_word'].apply(purpose_category)

Выделим категории по количеству детей('children'):

- Нет детей;
- 1 ребенок;
- 2 ребенка;
- Многодетные (>2 детей);

In [44]:
def children_category(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] < 2:
        return '1 ребенок'
    elif 2 <= row['children'] < 3:
        return '2 ребенка'
    else:
        return 'многодетные'
    
clientele['children_category'] = clientele.apply(children_category, axis=1)

Выделим категории по ежемесячному доходу('monthly_income'):

- Бедность (<= 50000р.);
- Средний достаток (50000 < x <= 120000);
- Состоятельные (120000 < x < 1000000);
- Богатые (x >= 1000000);

Разделение было проведено с учетом официальных категорий, на которые разделяется уровень жизни, по данным федеральной службы государственной статистики, и с поправкой на то, что в приведенном датафрейме зарплаты выше, чем средние значения категорий в целом. 

In [45]:
def monthly_income_category(row):
    if row['monthly_income'] <= 50000:
        return 'бедность'
    elif 50000 < row['monthly_income'] <= 120000:
        return 'средний достаток'
    elif 120000 < row['monthly_income'] < 1000000:
        return 'состоятельные'
    else:
        return 'богатые'
    
clientele['monthly_income_category'] = clientele.apply(monthly_income_category, axis=1)

Разделение по возрасту было выполнено ранее. Разделение по стажу не проводилось, поскольку эти данные не будут использоваться непосредственно в ответах на вопросы.

In [46]:
clientele.head(15)

,children,days_employed,age,education,education_id,family_status,family_status_id,gender,income_type,loan_delinquency,monthly_income,purpose,age_category,purpose_word,purpose_category,children_category,monthly_income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,31 - 45 лет,покупка жилье \n,жилье,1 ребенок,состоятельные
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,31 - 45 лет,приобретение автомобиль \n,автомобиль,1 ребенок,средний достаток
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,31 - 45 лет,покупка жилье \n,жилье,нет детей,состоятельные
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,31 - 45 лет,дополнительный образование \n,образование,многодетные,состоятельные
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,46 - 60 лет,сыграть свадьба \n,свадьба,нет детей,состоятельные
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,19 - 30 лет,покупка жилье \n,жилье,нет детей,состоятельные
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,31 - 45 лет,операция с жилье \n,жилье,нет детей,состоятельные
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,46 - 60 лет,образование \n,образование,нет детей,состоятельные
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,31 - 45 лет,на проведение свадьба \n,свадьба,2 ребенка,средний достаток
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,31 - 45 лет,покупка жилье для семья \n,жилье,нет детей,состоятельные


**Вывод**

Выполнена категоризация датафрейма по следующим данным:

- 'education_id' и 'family_status_id' - уже изначально были разбиты на категории;
- по возрасту ('age');
- по цели кредита ('purpose');
- по количеству детей ('children');
- по ежемесячному доходу ('monthly_income').

Все данные были структурированы с учетом следующих требований к категориям: существенность признака категоризации, его уникальность и потенциал наполнения.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для начала создадим сводную талицу, чтобы увидеть данные.

In [47]:
clientele_pivot1 = clientele.pivot_table(index=['loan_delinquency'], columns='children_category', values = 'age', aggfunc='count')
print(clientele_pivot1)
print()

children_category  1 ребенок  2 ребенка  многодетные  нет детей
loan_delinquency                                               
0                       4364       1926          349      13074
1                        444        202           31       1064



Теперь создадим фукцию для вывода результатов.

In [48]:
def results(category):
    return clientele.groupby(category)['loan_delinquency'].mean().to_frame().sort_values(by='loan_delinquency')
results('children_category')

,loan_delinquency
children_category,
нет детей,0.075258
многодетные,0.081579
1 ребенок,0.092346
2 ребенка,0.094925


**Вывод**

Можно сделать вывод, что наличие детей увеличивает шанс просрочки по кредиту. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим сводную таблицу, чтобы увидеть данные. После чего воспользуемся функцией для вывода результатов.

In [49]:
clientele_pivot2 = clientele.pivot_table(index=['loan_delinquency'], columns='family_status', values='age', aggfunc='count')
print(clientele_pivot2)
print()

results('family_status')

family_status     в разводе  вдовец / вдова  гражданский брак  \
loan_delinquency                                                
0                      1110             896              3763   
1                        85              63               388   

family_status     женат / замужем  не женат / не замужем  
loan_delinquency                                          
0                           11408                   2536  
1                             931                    274  



,loan_delinquency
family_status,
вдовец / вдова,0.065693
в разводе,0.071130
женат / замужем,0.075452
гражданский брак,0.093471
не женат / не замужем,0.097509


**Вывод**

Клиенты, которые не находятся (не находились) в узаконенных отношениях - гражданский брак или не женат / не замужем - реже выплачивают кредит в срок.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создадим сводную таблицу, после чего используем функцию для вывода результатов.

In [50]:
clientele_pivot3 = clientele.pivot_table(index=['loan_delinquency'], columns='monthly_income_category', values='age', aggfunc='count')
print(clientele_pivot3)
print()

results('monthly_income_category')

monthly_income_category  бедность  богатые  состоятельные  средний достаток
loan_delinquency                                                           
0                             349       23          13396              5945
1                              23        2           1188               528



,loan_delinquency
monthly_income_category,
бедность,0.061828
богатые,0.080000
состоятельные,0.081459
средний достаток,0.081570


**Вывод**

Интересное наблюдение: люди с наименьшим доходом чаще других выплачивают кредит в срок. При этом отличие данной группы от других больше, чем различия между оставшимися тремя. (1,9% между бедностью и богатыми, против ~0,1% между оставшимися)

- Как разные цели кредита влияют на его возврат в срок?

Создадим таблицу с данными, а после этого используем фукцию, чтобы увидеть результаты.

In [51]:
clientele_pivot3 = clientele.pivot_table(index=['loan_delinquency'], columns='purpose_category', values='age', aggfunc='count')
print(clientele_pivot3)
print()

results('purpose_category')

purpose_category  автомобиль  жилье  коммерческая недвижимость  образование  \
loan_delinquency                                                              
0                       3903   8245                       1212         3643   
1                        403    648                         99          370   

purpose_category  ремонт  свадьба  
loan_delinquency                   
0                    572     2138  
1                     35      186  



,loan_delinquency
purpose_category,
ремонт,0.057661
жилье,0.072866
коммерческая недвижимость,0.075515
свадьба,0.080034
образование,0.092200
автомобиль,0.093590


**Вывод**

Наиболее ответсвенные заемщики берут кредит на цели, связанные с жильем, лучше всех возвращают деньги те, кто взял их на ремонт.
А вот наиболее склонны нарушать выплаты те, кто брал кредит на автомобиль или образование.

## Шаг 4. Общий вывод

При работе с датафреймом были выполнены следующие задачи: 
- заполнение отсутсвующих значений;
- удаление дубликатов;
- корректировка артефактов; 
- обработка средних, медианных и эксремальных значений;
- категоризация данных.

Применив сводные таблицы, была проведена оценка влияния различных критериев на вероятность возврата кредита в срок.

Заключение.
Поставленная цель: "Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок". Да, действительно влияет: клиенты, которые не находятся в законных отношениях, чаще не выплачивают кредит в срок; вероятность того, что клиент просрочит платеж, выше в случае наличия детей.

Сделаем некоторое подобие портфолио для ответсвенных и наименее ответственных заемщиков.

Ответственный заемщик:
- не имеет детей
- был или находится в официальном браке
- имеет уровень ежемесячного дохода >50000
- берет кредит на операции с жильем, лучше всего на ремонт

Наименее ответсвенный заемщик:
- имеет 1 или 2 детей
- не женат / не замужем
- имеет средний уровень достатка
- берет кредит на автомобиль или обучение


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.